In [1]:
import pandas as pd
import numpy as np
from cataclop.ml.pipeline import factories

In [2]:
d = factories.Dataset.factory('default')

In [3]:
d.load(force=True)
#d.save()

In [6]:
d.players.head()


age  earnings  final_odds_ref  handicap_distance  handicap_weight  \
race_id id                                                                      
1       1     5  19810000             1.6             2100.0              NaN   
        2     5  15748400             5.8             2100.0              NaN   
        3     4  19096000            12.4             2100.0              NaN   
        4     4   7796500            21.5             2100.0              NaN   
        5     5  18067000            14.5             2100.0              NaN   

            herder_id  horse_id         imported_at_player  is_first_timer  \
race_id id                                                                   
1       1         1.0         1 2018-05-21 17:41:31.213933           False   
        2         NaN         2 2018-05-21 17:41:31.218841           False   
        3         2.0         3 2018-05-21 17:41:31.124658           False   
        4         3.0         4 2018-05-21 17:41:31.137362           False   
        5         4.0         5 2018-05-21 17:41:31.149920           False   

            is_racing    ...                start_at          sub_category  \
race_id id               ...                                                 
1       1        True    ...     2017-10-20 19:43:00  EUROPEENNE_AUTOSTART   
        2        True    ...     2017-10-20 19:43:00  EUROPEENNE_AUTOSTART   
        3        True    ...     2017-10-20 19:43:00  EUROPEENNE_AUTOSTART   
        4        True    ...     2017-10-20 19:43:00  EUROPEENNE_AUTOSTART   
        5        True    ...     2017-10-20 19:43:00  EUROPEENNE_AUTOSTART   

                  date  hippodrome_id        imported_at_session  num_session  \
race_id id                                                                      
1       1   2017-10-20              1 2018-05-21 17:41:31.070803            1   
        2   2017-10-20              1 2018-05-21 17:41:31.070803            1   
        3   2017-10-20              1 2018-05-21 17:41:31.070803            1   
        4   2017-10-20              1 2018-05-21 17:41:31.070803            1   
        5   2017-10-20              1 2018-05-21 17:41:31.070803            1   

            code  country          imported_at_hippo       name  
race_id id                                                       
1       1    VIN      FRA 2018-05-21 14:53:00.375766  VINCENNES  
        2    VIN      FRA 2018-05-21 14:53:00.375766  VINCENNES  
        3    VIN      FRA 2018-05-21 14:53:00.375766  VINCENNES  
        4    VIN      FRA 2018-05-21 14:53:00.375766  VINCENNES  
        5    VIN      FRA 2018-05-21 14:53:00.375766  VINCENNES  

[5 rows x 48 columns]

In [9]:
races = d.players.groupby('race_id')

stats = races[['race_count', 'victory_count', 'placed_2_count', 'placed_3_count']].agg([np.mean, np.std])
stats.columns = ['_'.join(col) for col in stats.columns.values]


df = d.players.join(stats, how='inner')

df['placed_2_count_r'] = df['placed_2_count'] - df['placed_2_count_mean'] / df['placed_2_count_std']
df['placed_3_count_r'] = df['placed_3_count'] - df['placed_3_count_mean'] / df['placed_3_count_std']
df['race_count_r'] = df['race_count'] - df['race_count_mean'] / df['race_count_std']
df['victory_count_r'] = df['victory_count'] - df['victory_count_mean'] / df['victory_count_std']

nulls = df[['race_count_r', 'victory_count_r']].isnull().max(axis=1)
#print(nulls==True)
df.drop(nulls.loc[nulls==True].index, inplace=True)

odds = pd.DataFrame(columns=['odds_{:d}'.format(i) for i in range(20)], index=df.index)
odds.fillna(-1000)

#print(df)

races = df.groupby('race_id')

for (id, race) in races:
    odds_sorted = sorted(race['final_odds_ref'].values)
#    print(odds_sorted, race)
    odds.loc[race.index, ['odds_{:d}'.format(i) for i, v in enumerate(odds_sorted)]] = odds_sorted

df = pd.concat([df,odds], axis=1)

df[['odds_{:d}'.format(i) for i in range(20)]] = df[['odds_{:d}'.format(i) for i in range(20)]].fillna(-1000.0)

df.head()


age  earnings  final_odds_ref  handicap_distance  handicap_weight  \
race_id id                                                                      
1       1     5  19810000             1.6             2100.0              NaN   
        2     5  15748400             5.8             2100.0              NaN   
        3     4  19096000            12.4             2100.0              NaN   
        4     4   7796500            21.5             2100.0              NaN   
        5     5  18067000            14.5             2100.0              NaN   

            herder_id  horse_id         imported_at_player  is_first_timer  \
race_id id                                                                   
1       1         1.0         1 2018-05-21 17:41:31.213933           False   
        2         NaN         2 2018-05-21 17:41:31.218841           False   
        3         2.0         3 2018-05-21 17:41:31.124658           False   
        4         3.0         4 2018-05-21 17:41:31.137362           False   
        5         4.0         5 2018-05-21 17:41:31.149920           False   

            is_racing   ...     odds_10  odds_11  odds_12  odds_13  odds_14  \
race_id id              ...                                                   
1       1        True   ...         NaN      NaN      NaN      NaN      NaN   
        2        True   ...         NaN      NaN      NaN      NaN      NaN   
        3        True   ...         NaN      NaN      NaN      NaN      NaN   
        4        True   ...         NaN      NaN      NaN      NaN      NaN   
        5        True   ...         NaN      NaN      NaN      NaN      NaN   

            odds_15  odds_16  odds_17  odds_18  odds_19  
race_id id                                               
1       1       NaN      NaN      NaN      NaN      NaN  
        2       NaN      NaN      NaN      NaN      NaN  
        3       NaN      NaN      NaN      NaN      NaN  
        4       NaN      NaN      NaN      NaN      NaN  
        5       NaN      NaN      NaN      NaN      NaN  

[5 rows x 80 columns]

In [14]:
df.index.droplevel(1).values

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, 14, 14, 14, 14, 14, 14, 14, 16,
       16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26,
       26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27,
       27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
       28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31,
       31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32,
       32, 32, 32, 32, 32, 32, 32])

In [15]:
from sklearn.model_selection import KFold, GroupKFold

groups = df.index.droplevel(1).values

group_kfold = GroupKFold(n_splits=3)

In [19]:
df.head()

age  earnings  final_odds_ref  handicap_distance  handicap_weight  \
race_id id                                                                      
1       1     5  19810000             1.6             2100.0              NaN   
        2     5  15748400             5.8             2100.0              NaN   
        3     4  19096000            12.4             2100.0              NaN   
        4     4   7796500            21.5             2100.0              NaN   
        5     5  18067000            14.5             2100.0              NaN   

            herder_id  horse_id         imported_at_player  is_first_timer  \
race_id id                                                                   
1       1         1.0         1 2018-05-21 17:41:31.213933           False   
        2         NaN         2 2018-05-21 17:41:31.218841           False   
        3         2.0         3 2018-05-21 17:41:31.124658           False   
        4         3.0         4 2018-05-21 17:41:31.137362           False   
        5         4.0         5 2018-05-21 17:41:31.149920           False   

            is_racing  ...   odds_11  odds_12  odds_13  odds_14  odds_15  \
race_id id             ...                                                 
1       1        True  ...       NaN      NaN      NaN      NaN      NaN   
        2        True  ...       NaN      NaN      NaN      NaN      NaN   
        3        True  ...       NaN      NaN      NaN      NaN      NaN   
        4        True  ...       NaN      NaN      NaN      NaN      NaN   
        5        True  ...       NaN      NaN      NaN      NaN      NaN   

            odds_16  odds_17  odds_18  odds_19  pred  
race_id id                                            
1       1       NaN      NaN      NaN      NaN   0.0  
        2       NaN      NaN      NaN      NaN   0.0  
        3       NaN      NaN      NaN      NaN   0.0  
        4       NaN      NaN      NaN      NaN   0.0  
        5       NaN      NaN      NaN      NaN   0.0  

[5 rows x 81 columns]

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

features = ['race_count_r', 'victory_count_r', 'placed_2_count_r', 'placed_3_count_r']

features = features + ['odds_{:d}'.format(i) for i in range(20)]

df['pred'] = 0.0

print(df[features].head())
    
for train_index, test_index in group_kfold.split(df.values, df['position'].values, groups):
    X_train = df[features].iloc[train_index].values
    y_train = (df['position'] == 1).iloc[train_index].astype('int32').values
    
    X_test = df[features].iloc[test_index].values
    y_test = (df['position'] == 1).iloc[test_index].astype('int32').values
    
    clf = RandomForestClassifier(n_estimators=20)
    clf = clf.fit(X_train, y_train)
    
    p = clf.predict_proba(X_test)
    
    print( log_loss(y_test, p) )
    
    idx = df.iloc[test_index].index
    
    df.loc[idx, 'pred'] = p[:, list(clf.classes_).index(1)]
    
    
    

            race_count_r  victory_count_r  placed_2_count_r  placed_3_count_r  \
race_id id                                                                      
1       1      22.299738         3.124853          4.533333            -1.218   
        2      12.299738         1.124853         -0.466667             1.782   
        3      26.299738         1.124853          3.533333             1.782   
        4      19.299738         1.124853         -0.466667            -0.218   
        5      56.299738         1.124853          3.533333             4.782   

            odds_0  odds_1  odds_2  odds_3  odds_4  odds_5   ...     odds_10  \
race_id id                                                   ...               
1       1      1.6     5.8    11.8    12.4    14.5    19.1   ...     -1000.0   
        2      1.6     5.8    11.8    12.4    14.5    19.1   ...     -1000.0   
        3      1.6     5.8    11.8    12.4    14.5    19.1   ...     -1000.0   
        4      1.6     5.8    11

In [26]:
df[['num', 'position', 'final_odds_ref', 'pred']][0:100]



num  position  final_odds_ref  pred
race_id id                                      
1       1      1       1.0             1.6  0.20
        2      2       2.0             5.8  0.00
        3      3       5.0            12.4  0.20
        4      4       NaN            21.5  0.15
        5      5       6.0            14.5  0.15
        6      6       7.0            36.1  0.05
        7      7       NaN            11.8  0.05
        8      8       4.0            48.7  0.10
        9      9       3.0            19.1  0.20
14      51     1       2.0             9.7  0.15
        52     2       4.0             6.5  0.20
        53     3       5.0             4.9  0.10
        54     4       6.0            18.1  0.10
        55     5       1.0             3.8  0.10
        56     6       7.0            37.4  0.10
        57     7       3.0             2.6  0.15
16      58     1       7.0            19.1  0.05
        59     2       1.0             1.1  0.10
        60     3       2.0            72.9  0.10
        61     4       6.0            18.4  0.10
        62     5       4.0            39.9  0.10
        63     6       5.0            41.9  0.10
        64     7       3.0            28.8  0.00
18      65     1       1.0             4.2  0.05
        66     2       2.0             2.2  0.05
        67     3       3.0             7.2  0.25
        68     4       5.0            12.8  0.30
        69     5       7.0             6.7  0.20
        71     7       4.0            25.1  0.05
        72     8       6.0            11.1  0.05
...          ...       ...             ...   ...
26      133    5       2.0            12.3  0.20
        134    6       4.0             6.8  0.20
        135    7       5.0            17.9  0.15
        136    8       3.0            10.0  0.00
        137    9       8.0            11.8  0.10
        138   10      11.0            11.8  0.10
        139   11       6.0             8.4  0.05
        140   12      10.0            14.7  0.20
        141   13      12.0            32.9  0.10
        142   14       1.0            15.2  0.05
27      143    1       5.0            14.7  0.20
        144    2      11.0            24.0  0.10
        145    3       2.0             8.6  0.25
        146    4       3.0             4.8  0.00
        147    5       4.0             6.5  0.00
        148    6       7.0             4.0  0.15
        149    7       1.0             8.4  0.00
        150    8       9.0            12.6  0.20
        151    9      10.0            23.3  0.20
        152   10       8.0            15.4  0.00
        153   11       6.0            28.4  0.00
28      154    1       6.0             4.4  0.00
        155    2       2.0            11.8  0.00
        156    3       8.0            13.2  0.00
        157    4      12.0            11.4  0.00
        158    5       3.0             4.8  0.00
        159    6       9.0            18.6  0.00
        160    7       7.0            12.1  0.05
        161    8      14.0            21.2  0.05
        162    9       1.0             8.5  0.00

[100 rows x 4 columns]

In [34]:
fi = pd.Series(clf.feature_importances_, index=features)
print(fi.sort_values(ascending=False))


race_count_r        0.247275
victory_count_r     0.229467
placed_2_count_r    0.185500
placed_3_count_r    0.165989
odds_5              0.047251
odds_0              0.033167
odds_3              0.015160
odds_2              0.014554
odds_1              0.014542
odds_6              0.012433
odds_4              0.010698
odds_14             0.007413
odds_7              0.004651
odds_13             0.003769
odds_8              0.002853
odds_9              0.002590
odds_10             0.002208
odds_12             0.000481
odds_18             0.000000
odds_11             0.000000
odds_15             0.000000
odds_16             0.000000
odds_17             0.000000
odds_19             0.000000
dtype: float64
